In [1]:
#!pip install sklearn
#pip install numpy
from sklearn import datasets
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#irisデータセットをダウンロード
iris = datasets.load_iris()
#3,4番目の特徴を抽出 追記:[2:3]とやることの違いが何なのかは、試しにやってみる
X = iris.data[:,[2,3]] #説明変数の中の　(全ての行、2，3列目)のデータをとる
y = iris.target #目的変数をとってくる
#目的変数の数を取得。(つまり、目的変数が何種類あるか調べる)
print('Class label:',np.unique(y))#今回のデータセットは3種類

Class label: [0 1 2]


In [ ]:
from sklearn.model_selection import train_test_split#訓練データとテストデータをデータセットを分割して作ってくれる

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1,stratify=y)
"""
全体の3割をテストデータにする。
random_stateを指定して乱数シードを固定すると常に同じように分割されることで、再現性を持ち、機械学習のモデルの性能を比較できるようにする。
stratifyを指定すると、目的変数が複数のラベルを持つ場合でも、各ラベルのデータ数が等しくなるようにデータを分割してくれる。
"""

'\n全体の3割をテストデータにする。\nrandom_stateを指定して乱数シードを固定すると常に同じように分割されることで、再現性を持ち、機械学習のモデルの性能を比較できるようにする。\nstratifyを指定すると、目的変数が複数のラベルを持つ場合でも、各ラベルのデータ数が等しくなるようにデータを分割してくれる。\n'

In [ ]:
print("Label counts in y:",np.bincount(y))#bincount():配列の中で自然数が何回登場したかを自然数ごとにカウントできる。ヒストグラム創る際にも使えそう
print("Label counts in y_train:",np.bincount(y_train))
print("Label counts in y_test:",np.bincount(y_test))
#実行結果から、各ラベルのデータ数が等しくなるようにデータを分割されていることがわかる

Label counts in y: [50 50 50]
Label counts in y_train: [35 35 35]
Label counts in y_test: [15 15 15]


In [ ]:
#勾配降下法において、最小値を見つける際の処理が素早くなるように、標準化する
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)#訓練データの平均と標準偏差を計算

X_train_std = sc.transform(X_train)#平均と標準偏差を利用して、X_train を標準化する
X_test_std = sc.transform(X_test)#平均と標準偏差を利用して、X_test を標準化する

In [ ]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(max_iter = 40 ,eta0=0.1,random_state=1)#エポック数40,学習率0.1で設定

ppn.fit(X_train_std,y_train)

Perceptron(eta0=0.1, max_iter=40, random_state=1)

In [ ]:
y_pred = ppn.predict(X_test_std)#テストデータで予測

In [ ]:
print('Missclassified examples: %d' % (y_test != y_pred).sum()) #C言語でいう printf(%d,n)をpythonでは print("%d" % n)　であらわす（nは整数）
#今回の予測では、1つ誤分類したことがわかる

Missclassified examples: 1


In [ ]:
from sklearn.metrics import accuracy_score#分類の正解率を表示してくれる
print('Accuracy: %.3f' % accuracy_score(y_test,y_pred))

Accuracy: 0.978


In [ ]:
print('Accuracy: %.3f' % ppn.score(X_test_std,y_test))

Accuracy: 0.978


In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

In [ ]:
def plot_decision_regions(X,Y,classifier,test_idx=None,resolution=0.02):
  #マーカとカラーマップの準備
  markers = ("s","x","o","^","v") #それぞれ、s→四角形、x→バツ、o→円、^→上向き三角形、v→下向き三角形
  colors = ("red","blue","lightgreen","gray","cyan")#決定領域の色
  cmap = ListedColormap(colors[:len(np.unique(y))])#目的変数の数だけ、分類するための色を入手
  #決定領域のプロット
  x1_min,x1_max = X[:,0].min() - 1 , X[:,0].max() + 1 #今回は二値分類であり、二次元座標系に、データをプロットできる
  x2_min,x2_max = X[:,1].min() - 1 , X[:,1].max() + 1 #最小値からー1したところと最大値から＋1したところを表示する画面のサイズとすることで見やすくする
  #グリッドポイントの作成
  xx1,xx2 = np.meshgrid(np.arange(x1_min,x1_max, resolution),
                        np.arange(x2_min,))
  """
  np.meshgrid関数は、x, y, …の各座標の要素列から格子座標を作成するために使う　
  xx = np.array([[x for x in range(5)] for _ in range(5)])でも同じことができる
  第3引数はresolutionの幅でメッシュを描く
  https://deepage.net/features/numpy-meshgrid.html
  このサイトがイメージつきやすかった
  """
  #各特徴量を1次元配列に変換して予測を実行
  Z = classifier.predict(np.array([xx1.ravel(),xx2.ravel()]).T)#.Tは転置
  """
  # サンプルの二次元のリスト
  x =np.array([[1, 2, 3],[4, 5, 6]])

  # ravel関数を適用させると、一次元のリストが返却されます。
  print(np.ravel(x))
  [1 2 3 4 5 6]
  """
  #予測結果を元にグリッドポイントのデータサイズに変換
  z = Z.reshape(xx1.shape)#[xの要素数,1]に変換

  #グリッドポイントの等高線のプロット
  plt.contourf(xx1,xx2,Z,alpha=0.3,cmap=cmap)#contourf()関数は等高線を表示するメソッド  contourf(格子点のx座標の配列, 格子点のy座標の配列, 値の配列)
  #軸の範囲設定
  plt.xlim(xx1.min(),xx1.max())
  plt.ylim(xx2.min(),xx2.max())
  #クラスごとにデータをプロット
  for idx, cl in enumerate(np.unique(y)): #enumerate()はインデックス番号, 要素の順に取得できる。
    plt.scatter(x=X[y == cl,0],y=X[y==cl,1],alpha=0.8,c=colors[idx],marker=markers[idx],label=cl,edgecolor="black")

  #テストデータを目立たせる(点を〇で表示)
  if test_idx:
    #すべてのデータ点をプロット
    X_test,y_test = X[test_idx,:],y[test_idx,:]
    plt.scatter(X_test[:,0],X_test[:,1],c="",edgecolor="black",
                alpha=1.0,linewidth=1,marker="o",s=100,label="test set")


In [ ]:
from mlxtend.plotting import plot_decision_regions
#訓練データとテストデータの特徴量を行方向に結合
X_combined_std = np.vstack((X_train_std,X_test_std))
#訓練データとテストデータのクラスラベルを結合
y_combined = np.hstack((y_train,y_test))
#決定境界のプロット
plot_decision_regions(X=X_combined_std, y=y_combined, classifier=ppn, test_idx=range(105, 150))
#軸のラベルの設定
plt.xlabel("petal length [standardized]")
plt.ylabel("petal length [standardized]")
#凡例の設定(左上に配置)
plt.legend(loc="upper left")
#グラフの表示
plt.tight_layout()
plt.show()

SyntaxError: ignored